# 정리

### 데이터 load module

In [188]:
# 메인 데이터(log)
def main_data_load():
    import glob
    import gzip
    import re
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from collections import Counter
    import datetime
    

    # 옵션 설정
    plt.rcParams['font.family'] = 'NanumGothic' 
    pd.set_option('display.max.colwidth', 1000)

    pattern = r"\[(.*?)\] \[(.*?)\]: (.*)"
    file_list = glob.glob('log/*.gz')
    data = []
    for file in file_list:
        with gzip.open(file, 'rb') as f:
            log_data = f.read().decode('utf-8')
            groups = re.findall(pattern, log_data)
            for group in groups:
                data.append(group)

            
    df = pd.DataFrame(data, columns=['Timestamp', 'Level', 'Message'])
    df = df[~df['Message'].str.contains('청소')] # 청소 관련 처리 
    return df

# db 데이터
def db_data_load():
    import mysql.connector
    import pandas as pd
    db = # 정보
    cursor = db.cursor()
    query = 'SELECT * FROM player_info_view'
    cursor.execute(query)
    rows = cursor.fetchall()
    user_df = pd.DataFrame(rows, columns=['아이디','이름','계좌돈','은행레벨','유저돈','플레이어여부','등록날짜',
                                        '마지막로그인','전체게임수(도박)','승리횟수(도박)',
                                        '작은승리횟수(도박)','패배횟수(도박)'])
    lst = list(user_df['이름'].values) # 다른 데이터와 비교를 위해 
    return lst,user_df

## 전처리 공통 function

In [189]:
# 전처리 공통#

def user_logit(df,lst): # 메세지에서 유저 정보만 뽑아 데이터프레임으로 만드는 fuction fuction
    extracted = []
    for message in df['Message']:
        for ltem in lst:
            if ltem in message:
                extracted.append(ltem)
                break
        else:
            extracted.append(' ')
    df['user'] = extracted 
    return df 


def convert_to_seconds(timestamp): # timstamp 형식을 초로 환산하는 fuction
    import datetime
    time_obj = datetime.datetime.strptime(timestamp, '%H:%M:%S')
    seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    # 새벽 시간 (00:00:00 ~ 06:00:00) 처리
    if time_obj.hour < 6:
        seconds += 86400
    return seconds

### 유저분석

In [202]:
# 유저분석

def login_df_func():
    login_df = df[(df['Message'].str.contains('\[\+\].*')) | (df['Message'].str.contains('\[\-\].*'))] # 입,출입만
    login_df = login_df[~login_df['Message'].str.contains('님이')].reset_index().drop(['index'],axis=1) # 전처리
    login_df = login_df.drop([0, 2, 610])  # 이상 데이터 전처리 
    login_df['Seconds'] = login_df['Timestamp'].apply(convert_to_seconds) # 시간변환 
    return login_df

def time_df_func():
    time =[]
    for li in lst:
        plus = []
        minus = []
        for idx, message in enumerate(login_df['Message']):
            if (li in message) and ('[+]' in message):
                plus.append(login_df.iloc[idx]['Seconds'])
            elif (li in message) and ('[-]' in message):
                minus.append(login_df.iloc[idx]['Seconds'])

        # [+] 개수와 [-] 개수가 같은지 확인
        if len(plus) == len(minus):
            total_plus_time = sum(plus)
            total_minus_time = sum(minus)
            result = total_minus_time - total_plus_time
            time.append(result/3600)
        else:
            print(f"사용자 '{li}'의 [+] 개수와 [-] 개수가 일치하지 않습니다. plus 개수 : {len(plus)} , minus 개수 : {len(minus)}")
    return time

def user_info():
    top_users = login_df['Message'].str.extract(r'\[\w+\] ?(\w+)')[0].value_counts().head(5) # 지표1. 유저 로그인 순위 series
    # 지표 2. 유저별 누적시간 순위 데이터  
    time_df = pd.DataFrame({'User': lst , '누적 시간': time}).sort_values(by='누적 시간',ascending=False).reset_index().drop('index',axis=1).head(10)
    return top_users , time_df


def main():
    df = main_data_load()
    lst, user_df = db_data_load()
    login_df = login_df_func() 
    time = time_df_func()  
    top_users , time_df = user_info()  
    return df, lst, user_df, login_df, time , top_users ,time_df 

# 유저 접속 순위 / 유저 접속 시간 순위
df, lst, user_df, login_df, time , top_users ,time_df = main()



In [206]:
time_df

,User,누적 시간
0,kjs04,45.783333
1,WUSUCK,34.410000
2,taegyu08,26.010556
3,KDG_04,24.493056
4,chuzzang,23.718056
5,wooin04301,22.316389
6,biracksickhye,19.994444
7,hoyeonn_n,18.442778
8,Loitec1114,16.616667
9,hanu02,11.907500


## 구입 분석

In [182]:
# 구입분석 

def buy_df_func():
    buy_df = df[df['Message'].str.contains('구입')].copy()
    buy_df['Item'] = buy_df['Message'].str.extract(r'님이 ([^(]+)') # 님이 이후에 괄호를 뺀 모든 문자열 (아이템명)
    buy_df['Count'] = buy_df['Message'].str.extract(r'(\d+) 만큼') # 만큼 전 숫자 (몇개 샀는지)
    buy_df['cost'] = buy_df['Message'].str.extract(r'(\d+)를') # 를 전 숫자 (총 얼마인지) 
    buy_df["category"] = buy_df["Message"].str.findall(r"\((.+?)\.") # 몇개 샀는지 
    buy_df["category"] = buy_df["category"].apply(lambda x: ', '.join(x))
    buy_df = buy_df.astype({'Count': int, 'cost': int})
    buy_df = buy_df.reset_index().drop('index',axis=1)
    user_logit(buy_df,lst) # 유저 만 추출 
    return buy_df


def sell_df_multi_func(): # 한번에 여러상품 판경우
    sell_df_multi = df[df['Message'].str.contains('판매 gui')]
    sell_df_multi = sell_df_multi[~sell_df_multi['Message'].str.contains('챗')] # 이상치 제거 
    sell_df_multi['item'] = sell_df_multi['Message'].str.extractall(r'(\d+x\s([\w\s]+)\(.*?\))')[1].groupby(level=0).apply(list) # 여러개일수도 있으므로(아이템명)
    sell_df_multi['item'] = sell_df_multi['item'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x) 
    sell_df_multi['cost'] = sell_df_multi['Message'].str.extract(r'for (\d{1,3}(?:,\d{3})*(?:\.\d+)?)') # 총 비용 
    sell_df_multi['cost'] = sell_df_multi['cost'].str.replace(',', '').astype(float)
    sell_df_multi["category"] = sell_df_multi["Message"].str.findall(r"\((.+?)\.") # 카테고리 
    sell_df_multi["category"] = sell_df_multi["category"].apply(lambda x: ', '.join(x)) # 여러개일수도 있으므로
    sell_df_multi = sell_df_multi.reset_index().drop('index',axis=1)
    return sell_df_multi


def sell_df_one_func(): # 한번에 한종류 상품 판경우
    sell_df_one = df[df['Message'].str.contains('판매 하셨습니다')].copy()
    sell_df_one['item'] = sell_df_one['Message'].str.extract(r'님이 ([^(]+)') # 아이템명 
    sell_df_one['cost'] = sell_df_one['Message'].str.extract(r'(\d+)를') # 총 얼마인지 
    sell_df_one['cost'] = sell_df_one['cost'].astype(float) 
    sell_df_one["category"] = sell_df_one["Message"].str.findall(r"\((.+?)\.") # 카테고리 
    sell_df_one["category"] = sell_df_one["category"].apply(lambda x: ', '.join(x)) # 여러개일수도 있으므로
    sell_df_one = sell_df_one.reset_index().drop('index',axis=1)
    return sell_df_one

''' 참고로 한개 판 경우 count를 만들지 않은 이유는 여러개 여러개판경우와 합치기 위해서다. 
    나중에 counter함수를 이용해 물품별 판매 갯수를 충분히 셀 수 있다! 구매는 하나를 산 경우만 있으므로 따로 count를 만들어줌 '''

def shop_df_fuc():
    buy_df = buy_df_func()
    sell_df_multi = sell_df_multi_func()
    sell_df_one = sell_df_one_func()
    sell_df = pd.concat([sell_df_multi,sell_df_one])
    sell_df['item'] = sell_df['item'].str.replace(',\s*', ',', regex=True) # 공백제거
    sell_df['category'] = sell_df['category'].str.replace(',\s*', ',', regex=True) #공백제거
    user_logit(sell_df,lst)
    return buy_df , sell_df_multi , sell_df_one , sell_df


buy_df,sell_df_multi,sell_df_one,sell_df = shop_df_fuc()

def shop_info():
    buy_item_cost = buy_df.groupby('Item')['cost'].sum().sort_values(ascending=False).head(5) # 가장 산금액이 높은 아이템 
    buy_item_count = buy_df.groupby('Item')['Count'].sum().sort_values(ascending=False).head(5) # 가장 많이 산 아이템
    buy_item_category = buy_df.groupby('category')['Count'].sum().sort_values(ascending=False).head(5) # 가장 많이 산 카테고리

    king_user_buy = buy_df.groupby('user')['Count'].sum().sort_values(ascending=False).head(5) # 유저 구매왕
    user_buy_category = buy_df.groupby(['user','category'])['Count'].sum().sort_values(ascending=False).head(10) # 유저별 카테고리 구매 개수 
    user_buy_item = buy_df.groupby(['user','Item'])['Count'].sum().sort_values(ascending=False).head(10) # 유저별 어떤 아이템 많이 샀는지 

    sell_df_multi_cost = sell_df_multi.groupby('item')['cost'].sum().sort_values(ascending=False).head(5)  # 조합 최고 돈번 상품 
    sell_df_one_cost = sell_df_one.groupby('item')['cost'].sum().sort_values(ascending=False).head(5) # 한개만 캐서 파는 최고 돈번 상품 


    all_words = ",".join(sell_df["item"])
    word_counts = Counter(all_words.split(','))
    sell_count =  word_counts.most_common(10)
    sell_count_data = pd.DataFrame(sell_count, columns=['품목', '판매량']) # 가장 많이 판매한 아이템 


    all_words2 = ",".join(sell_df["category"])
    word_counts2 = Counter(all_words2.split(','))
    sell_count2 =  word_counts2.most_common(10)
    sell_count_data2 = pd.DataFrame(sell_count2, columns=['카테고리', '판매량']) # 가장 많이 판매한 카테고리 


    king_user_sell = sell_df.groupby('user')['cost'].sum().sort_values(ascending=False).head(5) # 유저 판매왕

    return buy_item_cost,buy_item_count,buy_item_category,sell_df_multi_cost,sell_df_one_cost,king_user_buy,user_buy_category,user_buy_item,sell_count_data,sell_count_data2,king_user_sell



buy_item_cost,buy_item_count,buy_item_category,sell_df_multi_cost,sell_df_one_cost,sell_count_data,sell_count_data,king_user_buy,user_buy_category,user_buy_item,king_user_sell = shop_info()

## 유저별 분석(채팅)

In [183]:
# 채팅 분석
def user_info_func():
    chat_df = df[df['Level'].str.contains('Async Chat Thread')]
    user_logit(chat_df,lst) # message에서 유저 생성
    chat_df_new = chat_df[chat_df['Message'].str.contains('[뉴비]')]  
    return chat_df , chat_df_new

chat_df , chat_df_new = user_info_func()

def uesr_info():
    top_chat = chat_df['user'].value_counts().head(5)
    top_chat_new = chat_df_new['user'].value_counts().head(5)
    return top_chat,top_chat_new

top_chat,top_chat_new = uesr_info()


C:\Users\administ\ipykernel_11792\2712225968.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user'] = extracted
